In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown
import sys
from tqdm import tqdm
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [16]:
df = pd.read_excel('/content/DomainExtract_20230616.xlsx')

In [19]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

In [20]:
df

,postcode,state,suburb_name,bedrooms,type,median_price,median_price_int,rental_median_price,rental_yield,avg_days_on_market,...,no_of_sales_2022,median_price_2021,growth_rate_2021,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019
0,0800,Northern Territory,Darwin City,2,House,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0800,Northern Territory,Darwin City,3,House,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0800,Northern Territory,Darwin City,4,House,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0800,Northern Territory,Darwin City,1,Unit,$261k,261000,0,0%,161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0800,Northern Territory,Darwin City,2,Unit,$415k,415000,0,0%,213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25126,7470,Tasmania,Rosebery,3,House,$161k,161000,0,0%,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25127,7470,Tasmania,Rosebery,4,House,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25128,7470,Tasmania,Rosebery,5,House,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25129,7470,Tasmania,Rosebery,2,Unit,-,0,0,0%,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
def convert_to_int(value):
    if value == '0.0':
        return 0
    elif value.endswith("%"):
        return float(value[:-1])
    elif value.endswith("k"):
        return int(float(value[1:-1]) * 1000)
    elif value.endswith("m"):
        return int(float(value[1:-1]) * 1000000)
    elif '.' in value:
        try:
            return float(value)
        except ValueError:
            return 0
    else:
        try:
            return int(value)
        except ValueError:
            return 0

def replace_with_zero(x):
    if x == '-' or pd.isnull(x):
        return 0
    else:
        return x

In [23]:
df['rental_yield'] = [convert_to_int(value) for value in df['rental_yield']]

In [24]:
df['rental_yield'].unique()

array([0., 1., 2., 3., 4., 7., 6.])

In [26]:
df.columns

Index(['postcode', 'state', 'suburb_name', 'bedrooms', 'type', 'median_price',
       'median_price_int', 'rental_median_price', 'rental_yield',
       'avg_days_on_market', 'clearance_rate', 'sold_this_year', 'population',
       'average_age', 'owner', 'renter', 'family', 'single', 'suburb',
       'time_stamp', 'auction_clearance', 'average_days_on_market',
       'median_price_2023', 'growth_rate_2023', 'no_of_sales_2023',
       'median_price_2022', 'growth_rate_2022', 'no_of_sales_2022',
       'median_price_2021', 'growth_rate_2021', 'no_of_sales_2021',
       'median_price_2020', 'growth_rate_2020', 'no_of_sales_2020',
       'median_price_2019', 'growth_rate_2019', 'no_of_sales_2019'],
      dtype='object')

In [33]:
df['clearance_rate'] = df['clearance_rate'].apply(lambda x: 0 if x == '-' else x)
df['clearance_rate'].unique()

array([0.  , 0.53, 0.62, 0.57, 0.46, 0.42, 0.91, 0.44, 0.49, 0.48, 0.5 ,
       0.67, 0.47, 0.4 , 0.71, 0.68, 0.69, 0.63, 0.51, 0.54, 0.59, 0.76,
       0.73, 0.65, 0.75, 0.45, 0.38, 0.56, 0.64, 0.55, 0.6 , 0.28, 0.77,
       0.83, 0.7 , 0.79, 0.41, 0.36, 0.74, 0.58, 0.84, 0.89, 0.82, 0.52,
       0.39, 0.35, 0.3 , 0.95, 0.61, 0.85, 0.43, 0.8 , 0.81, 0.78, 0.72,
       0.88, 0.27, 0.9 , 0.33, 0.37, 0.66, 0.86, 0.25, 0.15, 0.2 , 0.92,
       0.17, 0.31, 0.34, 0.22, 0.94, 0.18, 0.19, 0.23, 1.  , 0.08, 0.32,
       0.29, 0.24, 0.26, 0.87, 0.21, 0.14, 0.93, 0.13, 0.16, 0.05, 0.09,
       0.06])

In [36]:
int_cols = ['bedrooms', 'median_price_int', 'rental_median_price', 'rental_yield',
       'avg_days_on_market', 'clearance_rate', 'sold_this_year', 'population',
       'owner', 'renter', 'family', 'single', 'auction_clearance', 'average_days_on_market',
       'median_price_2023', 'growth_rate_2023', 'no_of_sales_2023',
       'median_price_2022', 'growth_rate_2022', 'no_of_sales_2022',
       'median_price_2021', 'growth_rate_2021', 'no_of_sales_2021',
       'median_price_2020', 'growth_rate_2020', 'no_of_sales_2020',
       'median_price_2019', 'growth_rate_2019', 'no_of_sales_2019']

In [45]:
for cols in int_cols:
  df[cols] = [replace_with_zero(value) for value in df[cols]]
  df[cols] = df[cols].astype('str')
  df[cols] = [convert_to_int(value) for value in df[cols]]

In [52]:
df.columns

Index(['postcode', 'state', 'suburb_name', 'bedrooms', 'type', 'median_price',
       'median_price_int', 'rental_median_price', 'rental_yield',
       'avg_days_on_market', 'clearance_rate', 'sold_this_year', 'population',
       'average_age', 'owner', 'renter', 'family', 'single', 'suburb',
       'time_stamp', 'auction_clearance', 'average_days_on_market',
       'median_price_2023', 'growth_rate_2023', 'no_of_sales_2023',
       'median_price_2022', 'growth_rate_2022', 'no_of_sales_2022',
       'median_price_2021', 'growth_rate_2021', 'no_of_sales_2021',
       'median_price_2020', 'growth_rate_2020', 'no_of_sales_2020',
       'median_price_2019', 'growth_rate_2019', 'no_of_sales_2019'],
      dtype='object')

In [53]:
new_column_names = {
    'time_stamp':'timestamp',
    'suburb': 'domain_search_string'
}
df = df.rename(columns=new_column_names)

In [55]:
df.columns

Index(['postcode', 'state', 'suburb_name', 'bedrooms', 'type', 'median_price',
       'median_price_int', 'rental_median_price', 'rental_yield',
       'avg_days_on_market', 'clearance_rate', 'sold_this_year', 'population',
       'average_age', 'owner', 'renter', 'family', 'single',
       'domain_search_string', 'timestamp', 'auction_clearance',
       'average_days_on_market', 'median_price_2023', 'growth_rate_2023',
       'no_of_sales_2023', 'median_price_2022', 'growth_rate_2022',
       'no_of_sales_2022', 'median_price_2021', 'growth_rate_2021',
       'no_of_sales_2021', 'median_price_2020', 'growth_rate_2020',
       'no_of_sales_2020', 'median_price_2019', 'growth_rate_2019',
       'no_of_sales_2019'],
      dtype='object')

In [56]:
df.to_excel('DomainExtract_20230616.xlsx',index = False)